# Financial Modeling with ESG Data
Pulling company financials from the FMP API and generating financial analysis beyond Stock Pricing.

In [7]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import ipywidgets as widgets
from ipywidgets import interact
import yfinance as yf
load_dotenv()
api_key = os.getenv("fmp")
type(api_key)

str

In [8]:
#Function that returns income statement data
def get_data2(stock):
    data = requests.get(f"https://financialmodelingprep.com/api/v3/income-statement/{stock}?period=quarter&limit=400&apikey=FINANCIAL_API_KEY")

    data = data.json()
# Pull data from the previous for quarters. The 0 refers to the most # previous quarter etc. 
    data = data[0:5]
    
    return data

In [9]:
def get_tickers():
    wiki_page = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies').text
    sp_data = pd.read_html(wiki_page)
    ticker_df = sp_data[0]
    ticker_options = ticker_df['Symbol']
    return ticker_options

In [10]:
esg_data = pd.read_csv('Resources/ticker_data.csv')
ticker = esg_data['Symbol']
ticker

0        MMM
1        AOS
2        ABT
3       ABBV
4        ACN
5       ATVI
6        AYI
7       ADBE
8        AAP
9        AMD
10       AES
11       AET
12       AMG
13       AFL
14         A
15       APD
16      AKAM
17       ALK
18       ALB
19       ARE
20      ALXN
21      ALGN
22      ALLE
23       AGN
24       ADS
25       LNT
26       ALL
27     GOOGL
28      GOOG
29        MO
       ...  
475     VRTX
476     VIAB
477        V
478      VNO
479      VMC
480      WMT
481      WBA
482       WM
483      WAT
484      WEC
485      WFC
486      HCN
487      WDC
488       WU
489      WRK
490       WY
491      WHR
492      WMB
493     WLTW
494      WYN
495     WYNN
496      XEL
497      XRX
498     XLNX
499       XL
500      XYL
501      YUM
502      ZBH
503     ZION
504      ZTS
Name: Symbol, Length: 505, dtype: object

## Create data frame with esg info for stocks in csv

In [11]:
def web_scraper(ticker):
    elements = []
    web_data = requests.get('https://finance.yahoo.com/quote/'+ticker+'/sustainability?p='+ticker).text
    soup = BeautifulSoup(web_data, 'html.parser')
    esg_score = soup.find('div', {'class':'Fz(36px) Fw(600) D(ib) Mend(5px)'})
    datapoint = esg_score.text
    controversy_score = soup.find('div', {'class':'D(ib) Fz(36px) Fw(500)'})
    controversyz_datapoint = controversy_score.text
    scores = soup.find_all('div',{'class': 'D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)'})
    for score in scores:
            elements.append(score.text)
            
    df = pd.DataFrame({'Total ESG Score': datapoint,
                   'Environment Score': elements[0],
                   'Social Score': elements[1],
                   'Governance Score': elements[2],
                   'Controversy Score': controversy_datapoint},index=[ticker])
    df = df.astype('float')	
    df['Controversy Assessment'] = df.apply(lambda x: level(x['Controversy Score']), axis=1)
    return df

#print(web_data)

In [6]:
controversy_score = soup.find('div', {'class':'D(ib) Fz(36px) Fw(500)'})
controversy_datapoint = controversy_score.text
df = pd.DataFrame({'Total ESG Score': datapoint,
                   'Environment Score': elements[0],
                   'Social Score': elements[1],
                   'Governance Score': elements[2],
                   'Controversy Score': controversy_datapoint},
                  index=['ticker'])
df

NameError: name 'soup' is not defined

### Use Financial Modeling Prep API to gather financial data on companies listed in ESG search
#### -revenue, 
#### -net income, 
#### -earning per share growth, and 
#### -EBITDA growth.

In [ ]:
 def growth(stock):
    #call the get_data2 function and store the results in a variable, data 2.
    data = get_data2(stock)
    
    #revenue growth
    rev_new = data[0]['revenue']
    rev_old = data[-1]['revenue']
    rev_change = 100*((rev_new-rev_old)/rev_old)
    #net income growth
    income_new = data[0]['netIncome']
    income_old = data[-1]['netIncome']
    income_change = 100*((income_new-income_old)/income_old)
    
    #EPS growth
    eps_new = data[0]['eps']
    eps_old = data[-1]['eps']
    eps_change = 100*((eps_new-eps_old)/eps_old)
    
    #EBITA
    ebitda_new = data[0]['ebitda']
    ebitda_old = data[-1]['ebitda']
    ebitda_change = 100*((ebitda_new-ebitda_old)/ebitda_old)
    
    return (rev_change, income_change, eps_change, ebitda_change)


In [ ]:
stocks = [ticker]

#store financials in a variable, data.
data = map(growth, stocks)

# populate DataFrame with data variable and create columns:
df = pd.DataFrame(data, columns=['Rev_Growth', 'Income_Growth', 'EPS_Growth', 'ebitda_Growth'], index=stocks)
df.reset_index(inplace=True)
df.rename(columns={'index': 'Symbol'}, inplace=True)

#display output
df

In [ ]:
def dcf(stock):
    data = requests.get(f"https://financialmodelingprep.com/api/v3/discounted-cash-flow/{stock}?apikey=FINANCIAL_API_KEY")
    data = data.json()
    dcf = data[0]['dcf']
    current_price = data[0]['Stock Price']
    return (dcf, current_price)

In [ ]:
stocks = ['PTON', 'MSFT', 'AAPL', 'AMD']
data = map(dcf, stocks)
df = pd.DataFrame(data, columns=['dcf', 'current price'], index=stocks)
df.reset_index(inplace=True)
df.rename(columns={'index': 'symbol'}, inplace=True)
#Create a column that indicates under or over-valued
df['value'] = df.apply(lambda x: 'Under-valued' if x['dcf']>x['current price'] else 'Over-valued',axis=1)
#display results
df